## Imports

In [1]:
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

## Inits

In [2]:
use_gpu = torch.cuda.is_available()
nz = 100
ngf = 32
ndf = 64
nc = 3
batch_size = 64
beta1 = 0.5
image_size = 64
lr = 0.0002
epochs = 25
dataroot = "./svhn"
workers = 1
out_dir = "./dcgan_model"

## Dataset and data loader

In [3]:
dataset = dset.SVHN(
    root=dataroot,
    download=True,
    transform=transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]))

Using downloaded and verified file: ./svhn/train_32x32.mat


In [4]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

## Model

### Helpers

In [5]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

### Generator

In [6]:
class _netG(nn.Module):
    def __init__(self):
        super(_netG, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d(     nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 4 x 4
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 8 x 8
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 16 x 16
            nn.ConvTranspose2d(ngf * 2,     ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 32 x 32
            nn.ConvTranspose2d(    ngf,      nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 64 x 64
        )

    def forward(self, inputs):
        if isinstance(inputs.data, torch.cuda.FloatTensor) and use_gpu:
            output = nn.parallel.data_parallel(self.main, inputs, range(1))
        else:
            output = self.main(inputs)
        return output

In [7]:
netG = _netG()
netG.apply(weights_init)
print(netG)

_netG(
  (main): Sequential(
    (0): ConvTranspose2d (100, 256, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d (256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d (128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d (64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d (32, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


### Discriminator

In [8]:
class _netD(nn.Module):
    def __init__(self):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # input is (nc) x 64 x 64
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf) x 32 x 32
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*2) x 16 x 16
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*4) x 8 x 8
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # state size. (ndf*8) x 4 x 4
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, inputs):
        if isinstance(inputs.data, torch.cuda.FloatTensor) and use_gpu:
            output = nn.parallel.data_parallel(self.main, inputs, range(1))
        else:
            output = self.main(inputs)

        return output.view(-1, 1).squeeze(1)


In [9]:
netD = _netD()
netD.apply(weights_init)
print(netD)

_netD(
  (main): Sequential(
    (0): Conv2d (3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(0.2, inplace)
    (2): Conv2d (64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU(0.2, inplace)
    (5): Conv2d (128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU(0.2, inplace)
    (8): Conv2d (256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU(0.2, inplace)
    (11): Conv2d (512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)


### Criterion, labels, inputs, etc.

In [10]:
criterion = nn.BCELoss()

inputs = torch.FloatTensor(batch_size, 3, image_size, image_size)
noise = torch.FloatTensor(batch_size, nz, 1, 1)
fixed_noise = torch.FloatTensor(batch_size, nz, 1, 1).normal_(0, 1)
label = torch.FloatTensor(batch_size)
real_label = 1
fake_label = 0

if use_gpu:
    netD.cuda()
    netG.cuda()
    criterion.cuda()
    inputs, label = inputs.cuda(), label.cuda()
    noise, fixed_noise = noise.cuda(), fixed_noise.cuda()

fixed_noise = Variable(fixed_noise)

# setup optimizer
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))


## Training

In [11]:
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
for epoch in range(epochs):
    for i, data in enumerate(dataloader, 0):
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        # train with real
        netD.zero_grad()
        real_cpu, _ = data
        batch_size = real_cpu.size(0)
        if use_gpu:
            real_cpu = real_cpu.cuda()
        inputs.resize_as_(real_cpu).copy_(real_cpu)
        label.resize_(batch_size).fill_(real_label)
        inputv = Variable(inputs)
        labelv = Variable(label)

        output = netD(inputv)
        errD_real = criterion(output, labelv)
        errD_real.backward()
        D_x = output.data.mean()

        # train with fake
        noise.resize_(batch_size, nz, 1, 1).normal_(0, 1)
        noisev = Variable(noise)
        fake = netG(noisev)
        labelv = Variable(label.fill_(fake_label))
        output = netD(fake.detach())
        errD_fake = criterion(output, labelv)
        errD_fake.backward()
        D_G_z1 = output.data.mean()
        errD = errD_real + errD_fake
        optimizerD.step()

        ############################
        # (2) Update G network: maximize log(D(G(z)))
        ###########################
        netG.zero_grad()
        labelv = Variable(label.fill_(real_label))  # fake labels are real for generator cost
        output = netD(fake)
        errG = criterion(output, labelv)
        errG.backward()
        D_G_z2 = output.data.mean()
        optimizerG.step()

        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f D(x): %.4f D(G(z)): %.4f / %.4f'
              % (epoch, epochs, i, len(dataloader),
                 errD.data[0], errG.data[0], D_x, D_G_z1, D_G_z2))
        if i % 100 == 0:
            vutils.save_image(real_cpu,
                    '%s/real_samples.png' % out_dir,
                    normalize=True)
            fake = netG(fixed_noise)
            vutils.save_image(fake.data,
                    '%s/fake_samples_epoch_%03d.png' % (out_dir, epoch),
                    normalize=True)

    # do checkpointing
    torch.save(netG.state_dict(), '%s/netG_epoch_%d.pth' % (out_dir, epoch))
    torch.save(netD.state_dict(), '%s/netD_epoch_%d.pth' % (out_dir, epoch))

[0/25][0/1145] Loss_D: 1.8414 Loss_G: 3.4957 D(x): 0.3513 D(G(z)): 0.3885 / 0.0400
[0/25][1/1145] Loss_D: 0.9144 Loss_G: 5.8192 D(x): 0.8362 D(G(z)): 0.4577 / 0.0039
[0/25][2/1145] Loss_D: 0.4307 Loss_G: 5.5010 D(x): 0.8160 D(G(z)): 0.1625 / 0.0058
[0/25][3/1145] Loss_D: 0.3335 Loss_G: 4.8080 D(x): 0.8642 D(G(z)): 0.1206 / 0.0109
[0/25][4/1145] Loss_D: 0.3205 Loss_G: 5.1979 D(x): 0.9056 D(G(z)): 0.1668 / 0.0072
[0/25][5/1145] Loss_D: 0.4011 Loss_G: 7.0586 D(x): 0.9339 D(G(z)): 0.2576 / 0.0011
[0/25][6/1145] Loss_D: 0.3288 Loss_G: 5.9409 D(x): 0.8070 D(G(z)): 0.0539 / 0.0043
[0/25][7/1145] Loss_D: 0.2251 Loss_G: 5.7551 D(x): 0.9322 D(G(z)): 0.1211 / 0.0042
[0/25][8/1145] Loss_D: 0.2375 Loss_G: 6.4707 D(x): 0.9202 D(G(z)): 0.1245 / 0.0021
[0/25][9/1145] Loss_D: 0.1705 Loss_G: 7.0293 D(x): 0.9540 D(G(z)): 0.1100 / 0.0012
[0/25][10/1145] Loss_D: 0.1724 Loss_G: 6.1883 D(x): 0.9074 D(G(z)): 0.0548 / 0.0028
[0/25][11/1145] Loss_D: 0.2306 Loss_G: 8.1688 D(x): 0.9716 D(G(z)): 0.1662 / 0.0004
[0

[0/25][97/1145] Loss_D: 0.0108 Loss_G: 27.6310 D(x): 0.9914 D(G(z)): 0.0000 / 0.0000
[0/25][98/1145] Loss_D: 0.0006 Loss_G: 27.6310 D(x): 0.9994 D(G(z)): 0.0000 / 0.0000
[0/25][99/1145] Loss_D: 0.0003 Loss_G: 27.6310 D(x): 0.9997 D(G(z)): 0.0000 / 0.0000
[0/25][100/1145] Loss_D: 0.0004 Loss_G: 27.6310 D(x): 0.9996 D(G(z)): 0.0000 / 0.0000
[0/25][101/1145] Loss_D: 0.0002 Loss_G: 27.6309 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[0/25][102/1145] Loss_D: 0.0002 Loss_G: 27.6310 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[0/25][103/1145] Loss_D: 0.0002 Loss_G: 27.6309 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[0/25][104/1145] Loss_D: 0.0029 Loss_G: 27.6310 D(x): 0.9971 D(G(z)): 0.0000 / 0.0000
[0/25][105/1145] Loss_D: 0.0008 Loss_G: 27.6310 D(x): 0.9992 D(G(z)): 0.0000 / 0.0000
[0/25][106/1145] Loss_D: 0.0004 Loss_G: 27.6310 D(x): 0.9996 D(G(z)): 0.0000 / 0.0000
[0/25][107/1145] Loss_D: 0.0002 Loss_G: 27.6310 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[0/25][108/1145] Loss_D: 0.0010 Loss_G: 27.6310 D(x): 0.9

[0/25][193/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][194/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][195/1145] Loss_D: 0.0002 Loss_G: 27.6310 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[0/25][196/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][197/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][198/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][199/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][200/1145] Loss_D: 0.0002 Loss_G: 27.6310 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[0/25][201/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][202/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][203/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][204/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 

[0/25][289/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][290/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][291/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][292/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][293/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][294/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][295/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][296/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][297/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][298/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][299/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][300/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 

[0/25][385/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][386/1145] Loss_D: 0.0001 Loss_G: 27.6309 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][387/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][388/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][389/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][390/1145] Loss_D: 0.0002 Loss_G: 27.6310 D(x): 0.9998 D(G(z)): 0.0000 / 0.0000
[0/25][391/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][392/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][393/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][394/1145] Loss_D: 0.0001 Loss_G: 27.6310 D(x): 0.9999 D(G(z)): 0.0000 / 0.0000
[0/25][395/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[0/25][396/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 

...

[1/25][7/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][8/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][9/1145] Loss_D: 0.0000 Loss_G: 27.6308 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][10/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][11/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][12/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][13/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][14/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][15/1145] Loss_D: 0.0000 Loss_G: 27.6308 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][16/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][17/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][18/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)):

[1/25][105/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][106/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][107/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][108/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][109/1145] Loss_D: 0.0000 Loss_G: 27.6306 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][110/1145] Loss_D: 0.0000 Loss_G: 27.6308 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][111/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][112/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][113/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][114/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][115/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][116/1145] Loss_D: 0.0000 Loss_G: 27.6308 D(x): 

[1/25][201/1145] Loss_D: 0.0000 Loss_G: 27.6304 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][202/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][203/1145] Loss_D: 0.0000 Loss_G: 27.6310 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][204/1145] Loss_D: 0.0000 Loss_G: 27.6308 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][205/1145] Loss_D: 0.0000 Loss_G: 27.6306 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][206/1145] Loss_D: 0.0000 Loss_G: 27.6306 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][207/1145] Loss_D: 0.0000 Loss_G: 27.6308 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][208/1145] Loss_D: 0.0000 Loss_G: 27.6306 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][209/1145] Loss_D: 0.0000 Loss_G: 27.6309 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][210/1145] Loss_D: 0.0000 Loss_G: 27.6308 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][211/1145] Loss_D: 0.0000 Loss_G: 27.6304 D(x): 1.0000 D(G(z)): 0.0000 / 0.0000
[1/25][212/1145] Loss_D: 0.0000 Loss_G: 27.6306 D(x): 

Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes()
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
KeyboardInterrupt


KeyboardInterrupt: 